# Import das libs que serão utilizadas para manipulação dos dados e geração do relatório

In [1]:
import pandas as pd
import matplotlib
from pylab import title, figure, xlabel, ylabel, xticks, bar, legend, axis, savefig
from fpdf import FPDF
import numpy as np

# Cria a variável data com a busca que deverá ser realizada

In [2]:
data = '22/06/2020'
data = pd.to_datetime(data)

# Leitura dos arquivos


In [3]:
#Lê o arquivo excel com os ativos do ativos pertencentes ao IBOVESPA
df_carteira_ibov = pd.read_excel('carteira_ibov.xlsx', skiprows=9)
atv_ibov = df_carteira_ibov['Redutor'].values

#leitura dos aquivos CSV
df_setores = pd.read_csv('setores.csv',sep=",",encoding="latin-1")

df_preco_acoes = pd.read_csv('preco_acoes.csv',encoding="latin-1")

#converte a coluna data para o formate datetype
df_preco_acoes['date'] = pd.to_datetime(df_preco_acoes['date'])

In [4]:
df_atv_ibov = pd.DataFrame(atv_ibov[:len(atv_ibov)-2],columns=["Atv_IBOVESPA"])

#cria uma nova estrutura com os respectivos tickers e setores de cada componente do IBOVESPA
new_df =  pd.merge(left=df_atv_ibov, right=df_setores, left_on='Atv_IBOVESPA', right_on='ticker')

#deleta a coluna com dois tickers
del new_df['Atv_IBOVESPA']

# Encontra o preço de fechamento para cada um dos ativos na data procurada

In [5]:
#encontra o preço de fechamento para a data procurada
new_df = pd.merge(left=new_df, right=df_preco_acoes[df_preco_acoes['date']==data], left_on='ticker', right_on='ticker')

# Encontra a variação diaria

In [6]:
from datetime import datetime, timedelta

#diminui um dia para procurar a data UTIL anterior
k = 1
test_df_empty = pd.merge(left=new_df, right=df_preco_acoes[df_preco_acoes['date']==data - timedelta(days=k)], left_on='ticker', right_on='ticker').empty
while(test_df_empty):
    k = k + 1
    test_df_empty = pd.merge(left=new_df, right=df_preco_acoes[df_preco_acoes['date']==data - timedelta(days=k)], left_on='ticker', right_on='ticker').empty
    
#realizar o inner join para buscar as datas equivalentes ao dia util anterior    
new_df = pd.merge(left=new_df, right=df_preco_acoes[df_preco_acoes['date']==data - timedelta(days=k)], left_on='ticker', right_on='ticker')

#divide o preço da cotação recente pelo preço do dia util anterior
new_df['var diaria'] = (new_df['price close_x']/new_df['price close_y'] - 1).apply('{:.2%}'.format)

#renomeia a coluna com preço de fechamento
new_df.rename(columns={'price close_x':'price close'}, inplace=True)

#deleta as colunas criadas desnecessariamente
del new_df['date_x']
del new_df['date_y']
del new_df['price close_y']

# Encontra a variação 12 meses

In [7]:
from dateutil.relativedelta import relativedelta
k = 0
test_df_empty = pd.merge(left=new_df, right=df_preco_acoes[df_preco_acoes['date']==data - relativedelta(months=12)], left_on='ticker', right_on='ticker').empty

#subtrai 12 meses da data final e caso nao seja um dia util, procura até encontrar o dia util mais próximo
while(test_df_empty):
    k = k + 1
    test_df_empty = pd.merge(left=new_df, right=df_preco_acoes[df_preco_acoes['date']==data - relativedelta(months=12) +timedelta(days=k)], left_on='ticker', right_on='ticker').empty
    
#realizar o inner join para buscar as datas equivalentes ao dia util anterior    
new_df = pd.merge(left=new_df, right=df_preco_acoes[df_preco_acoes['date']==data - relativedelta(months=12) +timedelta(days=k)], left_on='ticker', right_on='ticker')

#divide o preço da cotação recente pelo preço do dia util anterior
new_df['var 12 meses'] = (new_df['price close_x']/new_df['price close_y'] - 1).apply('{:.2%}'.format)

#renomeia a coluna com preço de fechamento
new_df.rename(columns={'price close_x':'price close'}, inplace=True)

#deleta as colunas criadas desnecessariamente
del new_df['price close_y']
del new_df['date']


# Encontrar a variação mes

In [8]:
#será necessário encontrar o último dia do mês do anterior para realizarmos a comparação do preço da equity nesse dia
#e no dia de hoje

mes = int(str(data)[5:7]) - 1
ano = int(str(data)[:4])

#se o mes for 12 tera q procurar o ultimo dia util da virada do ano
if mes == 1:
    ano = ano - 1
    mes = 12

#procura na lista de dias os valores procurados
for i in range(0,len(df_preco_acoes['date'].unique())):
    if (int(str(df_preco_acoes['date'].unique()[i])[5:7])) == mes and (int(str(df_preco_acoes['date'].unique()[i])[:4]) == ano):
        data_ult_mes = df_preco_acoes['date'].unique()[i]
        break
        
#realizar o inner join para buscar as datas equivalentes ao dia util anterior    
new_df = pd.merge(left=new_df, right=df_preco_acoes[df_preco_acoes['date']==data_ult_mes], left_on='ticker', right_on='ticker')

#divide o preço da cotação recente pelo preço do dia util anterior
new_df['var mes'] = (new_df['price close_x']/new_df['price close_y'] - 1).apply('{:.2%}'.format)

#renomeia a coluna com preço de fechamento
new_df.rename(columns={'price close_x':'price close'}, inplace=True)

#deleta as colunas criadas desnecessariamente
del new_df['price close_y']
del new_df['date']


# Encontrar a variação ano

In [9]:
#subtrair um ano e procurar o ultimo dia da virada daquele ano
ano = int(str(data)[:4]) - 1

#procura na lista de dias os valores procurados
for i in range(0,len(df_preco_acoes['date'].unique())):
    if (int(str(df_preco_acoes['date'].unique()[i])[:4]) == ano):
        data_ult_ano = df_preco_acoes['date'].unique()[i]
        break
        
#realizar o inner join para buscar as datas equivalentes ao dia util anterior    
new_df = pd.merge(left=new_df, right=df_preco_acoes[df_preco_acoes['date']==data_ult_ano], left_on='ticker', right_on='ticker')

#divide o preço da cotação recente pelo preço do dia util anterior
new_df['var ano'] = (new_df['price close_x']/new_df['price close_y'] - 1)

#renomeia a coluna com preço de fechamento
new_df.rename(columns={'price close_x':'price close'}, inplace=True)

df_max_min_var = new_df[new_df.ticker.isin(new_df.sort_values('var ano')['ticker'][:5].values)].append(new_df[new_df.ticker.isin(new_df.sort_values('var ano')['ticker'].tail(5).values)])

#deleta as colunas criadas desnecessariamente
del new_df['price close_y']
del new_df['date']

In [10]:
from IPython.display import display, HTML
display(HTML(new_df.to_html()))

,ticker,setor,price close,var diaria,var 12 meses,var mes,var ano
0,ABEV3,Alimentos,14.110000,1.95%,-20.87%,13.06%,-0.244242
1,AZUL4,Transporte,20.630000,-5.37%,-52.65%,44.47%,-0.646019
2,B3SA3,Serviços Financeiros,50.195871,-1.31%,31.88%,10.49%,0.200501
3,BBAS3,Bancos,33.240000,-2.06%,-34.79%,7.78%,-0.361874
4,BBDC3,Bancos,19.622422,-1.75%,-32.29%,10.44%,-0.361765
5,BBDC4,Bancos,21.560643,-3.49%,-33.51%,13.99%,-0.339344
6,BBSE3,Seguradoras,28.830000,0.14%,0.61%,14.04%,-0.164621
7,BEEF3,Alimentos,12.980000,-4.84%,63.27%,-3.06%,0.010903
8,BPAC11,Bancos,75.990000,5.54%,55.57%,55.59%,0.006437
9,BRAP4,Mineração,36.070000,0.03%,10.94%,2.33%,-0.057732


In [11]:
df_max_min_var['var ano'] = df_max_min_var['var ano'].apply('{:.2%}'.format)
df_max_min_var = df_max_min_var[['ticker','setor','var ano']]
display(HTML(df_max_min_var.to_html()))


,ticker,setor,var ano
1,AZUL4,Transporte,-64.60%
23,CVCB3,Entretenimento,-55.25%
29,EMBR3,Transporte,-57.17%
39,HGTX3,Consumo,-56.79%
42,IRBR3,Seguradoras,-68.93%
14,BTOW3,Consumo,62.90%
49,MGLU3,Consumo,48.75%
50,MRFG3,Alimentos,28.82%
71,VVAR3,Consumo,33.39%
72,WEGE3,Indústria,32.13%
